## Deliverable 2. Create a Customer Travel Destinations Map.

In [32]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [33]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Leeton,AU,-34.57,146.40,59.00,62,20,8.05,few clouds
1,1,Tuktoyaktuk,CA,69.45,-133.04,-0.40,84,90,12.75,overcast clouds
2,2,Bambous Virieux,MU,-20.34,57.76,73.99,73,20,12.75,few clouds
3,3,Ponta Do Sol,PT,32.67,-17.10,66.99,68,40,8.05,scattered clouds
4,4,Hilo,US,19.73,-155.09,71.60,83,90,6.93,light rain


In [34]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [35]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
5,5,Faanui,PF,-16.48,-151.75,78.24,82,84,16.15,broken clouds
8,8,Hithadhoo,MV,-0.60,73.08,83.97,76,97,13.87,moderate rain
9,9,Dicabisagan,PH,17.08,122.42,77.45,83,76,7.58,moderate rain
11,11,Puerto Baquerizo Moreno,EC,-0.90,-89.60,75.20,69,75,6.93,broken clouds
14,14,Pecos,US,31.42,-103.49,82.40,23,1,14.99,clear sky
16,16,Georgetown,MY,5.41,100.34,78.80,88,20,2.93,few clouds
26,26,Elizabeth City,US,36.29,-76.25,78.80,61,1,9.17,clear sky
34,34,Maraba,BR,-5.37,-49.12,87.80,66,75,4.70,broken clouds
44,44,Honiara,SB,-9.43,159.95,77.00,94,20,2.24,few clouds
52,52,Manga,BF,11.66,-1.07,84.45,18,90,6.42,overcast clouds


In [36]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        201
City           201
Country        199
Lat            201
Lng            201
Max Temp       201
Humidity       201
Cloudiness     201
Wind Speed     201
Description    201
dtype: int64

In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df=preferred_cities_df.dropna()
clean_cities_df.count()

City_ID        199
City           199
Country        199
Lat            199
Lng            199
Max Temp       199
Humidity       199
Cloudiness     199
Wind Speed     199
Description    199
dtype: int64

In [39]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Faanui,PF,78.24,-16.48,-151.75,
8,Hithadhoo,MV,83.97,-0.60,73.08,
9,Dicabisagan,PH,77.45,17.08,122.42,
11,Puerto Baquerizo Moreno,EC,75.20,-0.90,-89.60,
14,Pecos,US,82.40,31.42,-103.49,
16,Georgetown,MY,78.80,5.41,100.34,
26,Elizabeth City,US,78.80,36.29,-76.25,
34,Maraba,BR,87.80,-5.37,-49.12,
44,Honiara,SB,77.00,-9.43,159.95,
52,Manga,BF,84.45,11.66,-1.07,


In [40]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [42]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.count()

City          199
Country       199
Max Temp      199
Lat           199
Lng           199
Hotel Name    199
dtype: int64

In [43]:
clean_hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Faanui,PF,78.24,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
8,Hithadhoo,MV,83.97,-0.60,73.08,Scoop Guest House
9,Dicabisagan,PH,77.45,17.08,122.42,Nancy Montanes Bernardo
11,Puerto Baquerizo Moreno,EC,75.20,-0.90,-89.60,Hotel los Algarrobos
14,Pecos,US,82.40,31.42,-103.49,Knights Inn Pecos


In [44]:
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
#Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [47]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig
fig.savefig("analysis/PyBer_fare_summary.png")

Figure(layout=FigureLayout(height='420px'))